<a href="https://colab.research.google.com/github/ktynski/Marketing_Automations_Notebooks_With_GPT/blob/main/Automatic_Intent_Prediction_from_Keyword_%2B_Serps_with_Automatic_Article_Title_Description_Generation_Based_on_Intent_Predictions_(Public).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Install the required libraries:

In [ ]:
!pip install openai
!pip install google-search-results

2. Import required libraries

In [ ]:
import openai
import re
import pandas as pd
import requests
from serpapi import GoogleSearch

2. Define the intent rubric.

In [ ]:
topic = "Enter your topic here"
intents = [
    "Given the search engine result page for the search query, classify the user's intent into one of the following 5 categories: \n Informational: The user is looking for general information on a topic.\n Navigational: The user is looking to find a specific website. \n Transactional: The user is looking to complete a transaction, such as purchasing a product or booking a service. \n Commercial Investigation: The user is comparing products or services or researching options for a potential purchase. \n Local: The user is looking for information about a specific local business or service. \n Please provide a brief explanation of your reasoning, justify your classification decision, and estimate a confidence score (between 0 and 1) for each category, indicating how confident you are in your prediction."

]

3. Initialize the OpenAI API key and the model name:

In [ ]:
# Initialize the OpenAI API key
openai.api_key = "Your OpenAI api key"
model_engine = "text-davinci-003"

4. Define a function that takes the topic and generates a list of high-intent types of searches for that subtopic. The more you ask for the more API calls will be done, but feel free to change 10 to any number.

In [ ]:
def generate_subtopics(topic):
    completions = openai.Completion.create(
        engine=model_engine,
        prompt="Provide a list of 10 general search query categories related to " + topic + " that likely have high intent.",
        max_tokens=400,
        n=1,
        stop=None,
        temperature=0.7,
    )

    message = completions.choices[0].text
    subtopics = message.strip().split("\n")
    subtopics = "\n".join(subtopics)

    nonumbersubtopics= re.sub(r"^\d+\.", "", subtopics, flags=re.MULTILINE)

    subtopics = nonumbersubtopics.split("\n")
    print(subtopics)
    return subtopics

#subtopics = generate_subtopics("fishing")

5. Generate keywords for each subtopic. The more you ask for the more API calls will be done, but feel free to change 10 to any number.


In [ ]:
def generate_keywords(subtopic):
    completions = openai.Completion.create(
        engine=model_engine,
        prompt="Provide a List of 10 representative high intent search keywords for " + subtopic,
        max_tokens=400,
        n=1,
        stop=None,
        temperature=0.7,
    )

    message = completions.choices[0].text
    keywords = message.strip().split("\n")
    keywords = "\n".join(keywords)

    nonumbertext = re.sub(r"^\d+\.", "", keywords, flags=re.MULTILINE)

    keywords = nonumbertext.split("\n")

    print(keywords)
    return keywords

#keywords = generate_keywords("fishing tackle")

6. Get SerpAPI Data

In [ ]:
def get_serp_data(keyword):
    api_key = "Your SerpAPI api key"
    params = {
        "engine": "google",
        "q": keyword,
        "api_key": api_key
    }
    search = GoogleSearch(params)
    results = search.get_dict()
    serp_data = results["organic_results"]

    titles = ", ".join([title for title in [d.get("title", None) for d in serp_data] if title is not None])
    links = ", ".join([link for link in [d.get("link", None) for d in serp_data] if link is not None])
    displayed_results = ", ".join([str(displayed_result) for displayed_result in [d.get("displayed_results", None) for d in serp_data] if displayed_result is not None])
    snippet_highlighted_words = ", ".join([str(snippet_highlighted_word) for snippet_highlighted_word in [d.get("snippet_highlighted_words", None) for d in serp_data] if snippet_highlighted_word is not None])

    finalresult =  keyword + ", " + titles + ", " + links + ", " + displayed_results + ", " + snippet_highlighted_words
    print(finalresult)
    return finalresult




7. Define a function that takes a keyword and the intent rubric, and returns a dictionary with the scores for each category in the rubric:

In [ ]:
import concurrent.futures

def get_serp_data_concurrent(keyword):
    return get_serp_data(keyword)

def evaluate_keyword_concurrent(keyword, intent):
    keywordserps = None
    with concurrent.futures.ThreadPoolExecutor(max_workers=10) as executor:
        future_keywordserps = executor.submit(get_serp_data_concurrent, keyword)
        keywordserps = future_keywordserps.result()

    completions = openai.Completion.create(
        engine=model_engine,
       prompt=f"You are an expert at predicting search intent when given both the search phrase used, and the search results from Google for that phrase. Given the intent rubric: \n '{intents}' \n Please try the example below: \n Search Keyword: '{keyword}' \n SERP Data:: '{keywordserps}',\n Please provide the intent prediction as well as a brief overview of a description of the intent as you understand it from the SERP data. \n Provide the result in the following format: [Intent Category / Intent Confidence Score / Detailed Explanation] Intent Prediction:",
        max_tokens=800,
        n=1,
        stop=None,
        temperature=0.7,
    )

    message = completions.choices[0].text
    #first_word = message.strip().split(" ")[0]


    return message

# Generate the subtopics and keywords
subtopics = generate_subtopics(topic)
keyword_lists = [generate_keywords(subtopic) for subtopic in subtopics]

# Evaluate the keywords and create a dataframe
rows = []
with concurrent.futures.ThreadPoolExecutor(max_workers=32) as executor:
    future_to_row = {executor.submit(evaluate_keyword_concurrent, keyword, intent): (subtopic, keyword, intent) for i, subtopic in enumerate(subtopics) for keyword in keyword_lists[i] for intent in intents}
    for future in concurrent.futures.as_completed(future_to_row):
        subtopic, keyword, intent = future_to_row[future]
        score = future.result()
        rows.append([subtopic, keyword]+[score if x == intent else None for x in intents])

df = pd.DataFrame(rows, columns=["Subtopic", "Keyword", "Intent"])


# Export the dataframe to a CSV file
df.to_csv("Intent_Prediction.csv", index=False)



8. Check the Dataframe and make sure things look good:

In [ ]:
df

9. Generate Article Titles and Outlines Given the Intent

In [ ]:
import concurrent.futures
import re

def generate_outlines(intent):
    completions = openai.Completion.create(
        engine=model_engine,
        prompt=f"For the given intent descriptions: \n '{intent}' Provide an article title and article outline.",
        max_tokens=1500,
        n=1,
        stop=None,
        temperature=0.7,
    )

    message = completions.choices[0].text
    titleandoutline = message

    print(titleandoutline)
    return titleandoutline

df["Article Ideas"] = ""

# Create an executor and submit tasks to the executor
with concurrent.futures.ThreadPoolExecutor() as executor:
    future_to_index = {executor.submit(generate_outlines, row["Intent"]): index for index, row in df.iterrows()}
    for future in concurrent.futures.as_completed(future_to_index):
        index = future_to_index[future]
        article_idea = future.result()
        df.at[index, "Article Ideas"] = article_idea

# Save the updated dataframe to a file
df.to_csv("Intent_and_Article_Suggestions.csv", index=False)